In [ ]:
from pandas import date_range
from datetime import timedelta

import panel as pn
import holoviews as hv
from holoviews.operation.datashader import datashade, dynspread
from holoviews.streams import Params
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import tools as btools
from bokeh.io import curdoc
import param

# add parent directory to path
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import z3dio
import timeio
import z3d_directory
import util
import matplotlib
matplotlib.use('TkAgg')

In [ ]:
hv.notebook_extension('bokeh','matplotlib')
pn.extension()

In [ ]:
# plot settings
# brenderer = renderer('bokeh')
dynspread.max_px=20
dynspread.threshold=0.5
# shade.cmap="#30a2da" # to match HV Bokeh default

verbose_formatter = DatetimeTickFormatter()
verbose_formatter.microseconds = ['%f us']
verbose_formatter.milliseconds = ['%S.%3N s']
# verbose_formatter.milliseconds = ['%H:%M:%S.%3N']
verbose_formatter.seconds = ['%H:%M:%S']
verbose_formatter.minsec = ['%H:%M:%S']
verbose_formatter.minutes = ['%H:%M']
verbose_formatter.hourmin = ['%H:%M']
verbose_formatter.hours = ['%m/%d %H:%M']
verbose_formatter.days = ['%m/%d', '%a%d']
verbose_formatter.months = ['%m/%Y', '%b %Y']
verbose_formatter.years = ['%Y']

In [ ]:
def no_logo(plot, element):
    # Don't plot the bokeh logo in the toolbar
    plot.state.toolbar.logo = None

In [ ]:
def get_ts(filename):
    # Return a plottable holoviews object of the time series in filename
    try:
        z3d = z3dio.read_z3d(filename)
        sampling_rate = z3d['metadata']['A/D Rate']
        if 'TX.SENSE' in z3d['metadata']:
            # NOTE: this only ids channel 1 TX files
            station_type = 'TX'
        else:
            station_type = 'RX'
        # These items were in schedule_info for RX
        rx_station = z3d['metadata']['RX.STN']
        channel_station = z3d['metadata']['CH.STN']
        channel = z3d['metadata']['Channel']
        component = z3d['metadata']['CH.CMP']
        box_number = z3d['metadata']['Box number']
        card_number = z3d['metadata']['ChannelSerial']
        # summary['schedule_info'] = schedule_info
        if z3d['metadata']['CH.CMP'][0] == 'H':
            # mag field
            antenna_number = z3d['metadata']['CH.ANTSN']
        elif z3d['metadata']['CH.CMP'][0] == 'E':
            # electric field
            antenna_number = None
        elif z3d['metadata']['CH.CMP'][0] == 'R':
            # electric field, transmitter ref channel
            antenna_number = None
        else:
            raise ValueError('CH.CMP not recognized as either E or H component')
        if z3d['num_records']==0:
            valid = False
        else:
            [dt_start,dt_end] = timeio.get_start_and_end_times_mountain(z3d)

        dt_start_naive = dt_start.replace(tzinfo=None)
        dt_end_naive = dt_end.replace(tzinfo=None)
        # period = timedelta(seconds=1)/sampling_rate
        sampling_increment = timedelta(seconds=1./sampling_rate)
        dt_times = date_range(start=dt_start_naive, periods=len(z3d['data']),
                                 freq=sampling_increment, tz=None,
                                 normalize=False, name=None, closed=None)

        curve_data = z3d['data']
        title_str = '{} Station: {:.0f}, '+\
            'Channel {:.0f} ({}), {} to {} Mountain'
        title_str = title_str.format(station_type, channel_station,
                                     channel, component, dt_start_naive, dt_end_naive)
        ts_tools = ['save','pan','xwheel_zoom','box_zoom','undo','reset']
        curve = hv.Curve((dt_times, curve_data),label=title_str,
                         kdims=['Time'],vdims=['Signal, V']
                        )
        # dscurve = datashade(curve, cmap=["blue"]).opts(width=800)
        # dscurve = datashade(curve,name=title_str).opts(width=800)
        dscurve = dynspread(datashade(curve).opts(width=800,height=400,
                                                  xformatter=verbose_formatter,
                                                  default_tools=ts_tools))
        dsfinal=dscurve.opts(plot=dict(finalize_hooks=[no_logo]))
    except:
        dsfinal=hv.Curve([]).options(xaxis=None, yaxis=None, show_frame=False)
    return dsfinal

In [ ]:
class TS(param.Parameterized):
    init_filename = '/Users/andymcaliley/SUBTER_Data/BelleCreekMT/BC-RXdata_201810/SUB2191b/2018-10-07/13_44_fielduser_ZEN74/ZenRawData/ZEN74_CH1/Zen074Ch1_20181007-134511_016.z3d'
    filename = param.String(default='')
    
    @param.depends('filename')
    def view(self):
        dsf = get_ts(self.filename)
        return dsf

In [ ]:
def get_file(event):
    fn = util.getfile(initialdir=ts.filename,title='Select Z3D file')
    if fn != '':
        ts.filename = ''
        file_text.object = os.path.basename(fn)
        ts.filename = fn
    #dsf = get_ts(fn)

In [ ]:
ts = TS()
file_text = pn.pane.Markdown(os.path.basename(ts.filename))
button = pn.widgets.Button(name='Load z3d', button_type='primary',width=100)
button.param.watch(get_file, 'clicks')

In [ ]:
pn.Column(pn.Row(button,file_text),ts.view).servable()